In [25]:
import os
import numpy as np
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
from pysal.explore import inequality
from inequality.gini import Gini_Spatial
from pysal.lib import weights
import warnings

In [145]:
path = "D:/Dumps/GEE-WP Scores/E2SFCA/100m grids/grid_geoms/"
cities_cont = 'Addis Ababa|Bologna|Cochabamba|Damascus|Detroit|Dhaka|Ghent|Indore|Kampala|Memphis|Philadelphia|Seattle|Shijiazhuang|Tel Aviv|Washington DC'
ls_file = os.listdir(path)
ls_file = pd.Series(ls_file)[pd.Series(ls_file).str.contains(cities_cont)].tolist()
city_score = [gpd.read_file(path+i) for i in ls_file]

In [146]:
%%time
# For loop city/GINI
# First non-spatial GINI for the 50 cities.

# Queen contignuity
# Spatial GINI - value cell itself
# Weight is values of surrounding cells
# Near and far, with weights of 1 for near and 0 for far.

warnings.filterwarnings('ignore')
cities_gini = []
cities_spatialgini = []
for city in enumerate(city_score):
    tr_gini = []
    tr_spatialgini = []
    print(ls_file[city[0]].split('.')[0])
    for i in city[1].columns[city[1].columns.str.contains('Sc-access')]:
        float_df = city[1][['geometry','population',i]]

        gini = pd.DataFrame(float_df[i].value_counts()).sort_index()
        gini['pop'] = float_df.groupby(i)['population'].sum()
        gini = gini.reset_index()
        gini['frac_pop'] = gini['pop'] / gini['pop'].sum()
        gini['richer_frac'] = 1-gini['frac_pop'].cumsum()
        gini['ugs'] = (gini['index'] * gini['pop']) 
        gini['frac_ugs'] = gini.ugs/gini.ugs.sum()
        gini_score = 1-(gini.frac_ugs*(gini.frac_pop+2*gini.richer_frac)).sum()
        print('gini',i.rsplit(' ')[-1],round(gini_score,4))
        tr_gini.append(gini_score)
        
        wq = weights.Queen.from_dataframe(float_df)
        wq.transform = "B"
        spatial_gini = Gini_Spatial(float_df[i], wq).g
        print('spatial gini',i.rsplit(' ')[-1],round(spatial_gini,4))
        tr_spatialgini.append(spatial_gini)
    cities_gini.append(tr_gini)
    cities_spatialgini.append(tr_spatialgini)
    print(' ')

Addis Ababa
gini 300 0.9645
spatial gini 300 0.961
gini 600 0.9532
spatial gini 600 0.956
gini 1000 0.9357
spatial gini 1000 0.9479
 
Bologna
gini 300 0.899
spatial gini 300 0.9145
gini 600 0.8257
spatial gini 600 0.8688
gini 1000 0.7572
spatial gini 1000 0.8144
 
Cochabamba
gini 300 0.9599
spatial gini 300 0.9889
gini 600 0.9375
spatial gini 600 0.9839
gini 1000 0.9276
spatial gini 1000 0.9794
 
Damascus
gini 300 0.865
spatial gini 300 0.9032
gini 600 0.7816
spatial gini 600 0.8406
gini 1000 0.7046
spatial gini 1000 0.7819
 
Detroit
gini 300 0.9564
spatial gini 300 0.9789
gini 600 0.8992
spatial gini 600 0.9572
gini 1000 0.8692
spatial gini 1000 0.9415
 
Dhaka
gini 300 0.9762
spatial gini 300 0.9885
gini 600 0.9446
spatial gini 600 0.975
gini 1000 0.9253
spatial gini 1000 0.9656
 
Ghent
gini 300 0.8902
spatial gini 300 0.8536
gini 600 0.8326
spatial gini 600 0.7993
gini 1000 0.7724
spatial gini 1000 0.7414
 
Indore
gini 300 0.9592
spatial gini 300 0.9927
gini 600 0.9275
spatial gini 6

In [147]:
gini_df = pd.DataFrame(cities_gini, columns = ['Gini E-300','Gini E-600','Gini E-1000'])
spatialgini_df = pd.DataFrame(cities_spatialgini, columns = ['S-gini E-300','S-gini E-600','S-gini E-1000'])
comb_gini = pd.concat([gini_df, spatialgini_df], axis = 1)
comb_gini.index = [i.split('.')[0] for i in ls_file]
comb_gini

,Gini E-300,Gini E-600,Gini E-1000,S-gini E-300,S-gini E-600,S-gini E-1000
Addis Ababa,0.964462,0.953176,0.935703,0.961043,0.956048,0.947909
Bologna,0.898971,0.825690,0.757233,0.914486,0.868787,0.814382
Cochabamba,0.959894,0.937514,0.927601,0.988892,0.983907,0.979380
Damascus,0.865005,0.781642,0.704602,0.903175,0.840585,0.781919
Detroit,0.956386,0.899226,0.869200,0.978881,0.957151,0.941513
Dhaka,0.976152,0.944615,0.925289,0.988498,0.975045,0.965588
Ghent,0.890233,0.832552,0.772449,0.853565,0.799285,0.741378
Indore,0.959225,0.927493,0.910574,0.992653,0.986436,0.983081
Kampala,0.946395,0.876125,0.786631,0.948669,0.888083,0.813234
Memphis,0.990884,0.975204,0.959250,0.990219,0.981046,0.970930


In [148]:
comb_gini.to_csv('D:/Dumps/GEE-WP Scores/E2SFCA/gini_scores')

In [50]:
%%time
File1 = gpd.read_file("D:/Dumps/GEE-WP Scores/Gravity/100m grids/grid_geoms/gridscore_grav3_Bologna.gpkg")

CPU times: total: 11.7 s
Wall time: 11.8 s


In [141]:
path = "D:/Dumps/GEE-WP Scores/Gravity/100m grids/grid_geoms/"
cities_cont = 'Addis Ababa|Bologna|Cochabamba|Damascus|Detroit|Dhaka|Ghent|Indore|Kampala|Memphis|Philadelphia|Seattle|Shijiazhuang|Tel Aviv|Washington DC'
ls_file = os.listdir(path)
ls_file = pd.Series(ls_file)[pd.Series(ls_file).str.contains(cities_cont)].tolist()
ls_file = pd.Series(ls_file)[pd.Series(ls_file).str.contains('grav3')].tolist()
city_score = [gpd.read_file(path+i) for i in ls_file]

In [142]:
%%time
warnings.filterwarnings('ignore')
Gcities_gini = []
Gcities_spatialgini = []
for city in enumerate(city_score):
    Gtr_gini = []
    Gtr_spatialgini = []
    print(ls_file[city[0]].split('.')[0].split('_')[-1])
    for i in city[1][['tr_300','tr_600','tr_1000']]:
        float_df = city[1][city[1][i].isna() == False][['geometry','population',i]]

        gini = pd.DataFrame(float_df[i].value_counts()).sort_index()
        gini['pop'] = float_df.groupby(i)['population'].sum()
        gini = gini.reset_index()
        gini['frac_pop'] = gini['pop'] / gini['pop'].sum()
        gini['richer_frac'] = 1-gini['frac_pop'].cumsum()
        gini['ugs'] = (gini['index'] * gini['pop']) 
        gini['frac_ugs'] = gini.ugs/gini.ugs.sum()
        gini_score = 1-(gini.frac_ugs*(gini.frac_pop+2*gini.richer_frac)).sum()
        print('gini',i.rsplit('_')[-1],round(gini_score,4))
        Gtr_gini.append(gini_score)
        
        wq = weights.Queen.from_dataframe(float_df)
        wq.transform = "B"
        spatial_gini = Gini_Spatial(float_df[i], wq).g
        print('spatial gini',i.rsplit('_')[-1],round(spatial_gini,4))
        Gtr_spatialgini.append(spatial_gini)
    Gcities_gini.append(Gtr_gini)
    Gcities_spatialgini.append(Gtr_spatialgini)
    print(' ')

Addis Ababa
gini 300 0.9111
spatial gini 300 0.9137
gini 600 0.848
spatial gini 600 0.892
gini 1000 0.7713
spatial gini 1000 0.8637
 
Bologna
gini 300 0.7473
spatial gini 300 0.8277
gini 600 0.5283
spatial gini 600 0.7004
gini 1000 0.3679
spatial gini 1000 0.5708
 
Cochabamba
gini 300 0.8109
spatial gini 300 0.8573
gini 600 0.6693
spatial gini 600 0.7617
gini 1000 0.5764
spatial gini 1000 0.6958
 
Damascus
gini 300 0.8501
spatial gini 300 0.8849
gini 600 0.7319
spatial gini 600 0.7889
gini 1000 0.6242
spatial gini 1000 0.6975
 
Detroit
gini 300 0.8865
spatial gini 300 0.8907
gini 600 0.7119
spatial gini 600 0.7239
gini 1000 0.5219
spatial gini 1000 0.5306
 
Dhaka
gini 300 0.9088
spatial gini 300 0.9491
gini 600 0.7976
spatial gini 600 0.8949
gini 1000 0.728
spatial gini 1000 0.8551
 
Ghent
gini 300 0.8745
spatial gini 300 0.9285
gini 600 0.752
spatial gini 600 0.857
gini 1000 0.6349
spatial gini 1000 0.7702
 
Indore
gini 300 0.8715
spatial gini 300 0.9644
gini 600 0.7658
spatial gini 6

In [143]:
Ggini_df = pd.DataFrame(Gcities_gini, columns = ['Gini G-300','Gini G-600','Gini G-1000'])
Gspatialgini_df = pd.DataFrame(Gcities_spatialgini, columns = ['S-gini G-300','S-gini G-600','S-gini G-1000'])
Gcomb_gini = pd.concat([Ggini_df, Gspatialgini_df], axis = 1)
Gcomb_gini.index = [i.split('.')[0].split('_')[-1] for i in ls_file]
Gcomb_gini

,Gini G-300,Gini G-600,Gini G-1000,S-gini G-300,S-gini G-600,S-gini G-1000
Addis Ababa,0.911057,0.848007,0.771306,0.913739,0.892014,0.863672
Bologna,0.747258,0.528262,0.367894,0.827650,0.700428,0.570820
Cochabamba,0.810930,0.669348,0.576412,0.857317,0.761721,0.695814
Damascus,0.850065,0.731924,0.624246,0.884937,0.788907,0.697472
Detroit,0.886456,0.711931,0.521877,0.890735,0.723934,0.530604
Dhaka,0.908796,0.797551,0.727979,0.949132,0.894928,0.855093
Ghent,0.874543,0.752039,0.634870,0.928533,0.857021,0.770226
Indore,0.871493,0.765847,0.685111,0.964401,0.929940,0.900090
Kampala,0.939785,0.827870,0.670124,0.939077,0.837034,0.695210
Memphis,0.973398,0.922738,0.839329,0.968036,0.930004,0.859867


In [144]:
Gcomb_gini.to_csv('D:/Dumps/GEE-WP Scores/Gravity/gini_scores')

In [149]:
gini_comb = pd.concat([comb_gini, Gcomb_gini], axis = 1)
gini_comb.to_csv('D:/Dumps/GEE-WP Scores/gini_all.csv')
gini_comb

,Gini E-300,Gini E-600,Gini E-1000,S-gini E-300,S-gini E-600,S-gini E-1000,Gini G-300,Gini G-600,Gini G-1000,S-gini G-300,S-gini G-600,S-gini G-1000
Addis Ababa,0.964462,0.953176,0.935703,0.961043,0.956048,0.947909,0.911057,0.848007,0.771306,0.913739,0.892014,0.863672
Bologna,0.898971,0.825690,0.757233,0.914486,0.868787,0.814382,0.747258,0.528262,0.367894,0.827650,0.700428,0.570820
Cochabamba,0.959894,0.937514,0.927601,0.988892,0.983907,0.979380,0.810930,0.669348,0.576412,0.857317,0.761721,0.695814
Damascus,0.865005,0.781642,0.704602,0.903175,0.840585,0.781919,0.850065,0.731924,0.624246,0.884937,0.788907,0.697472
Detroit,0.956386,0.899226,0.869200,0.978881,0.957151,0.941513,0.886456,0.711931,0.521877,0.890735,0.723934,0.530604
Dhaka,0.976152,0.944615,0.925289,0.988498,0.975045,0.965588,0.908796,0.797551,0.727979,0.949132,0.894928,0.855093
Ghent,0.890233,0.832552,0.772449,0.853565,0.799285,0.741378,0.874543,0.752039,0.634870,0.928533,0.857021,0.770226
Indore,0.959225,0.927493,0.910574,0.992653,0.986436,0.983081,0.871493,0.765847,0.685111,0.964401,0.929940,0.900090
Kampala,0.946395,0.876125,0.786631,0.948669,0.888083,0.813234,0.939785,0.827870,0.670124,0.939077,0.837034,0.695210
Memphis,0.990884,0.975204,0.959250,0.990219,0.981046,0.970930,0.973398,0.922738,0.839329,0.968036,0.930004,0.859867
